In [ ]:
import boto3, botocore
from botocore.exceptions import ClientError
import os, time, json, io, zipfile
from datetime import date
from dotenv import load_dotenv


from misc import load_from_yaml, save_to_yaml
import iam, s3, lf, rds, vpc, ec2

from ec2 import ALL_IN_ONE_INBOUND_RULES,ALL_IN_ONE_OUTBOUND_RULES,tags

load_dotenv(".env")
# boto3.setup_default_session(profile_name="AMominNJ")

In [ ]:
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ['AWS_DEFAULT_REGION']
VPC_ID            = os.environ['AWS_DEFAULT_VPC']
SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
SUBNET_ID         = SUBNET_IDS[0]
print(SUBNET_IDS)

In [3]:
sts_client           = boto3.client('sts')
rds_client           = boto3.client('rds')
iam_client           = boto3.client('iam')
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
lakeformation_client = boto3.client('lakeformation')
stepfunctions_client = boto3.client('stepfunctions')
apigateway_client    = boto3.client('apigateway')
lsn_client           = boto3.client('lambda')
events_client        = boto3.client('events')

In [4]:
ec2_client   = boto3.client('ec2', region_name=REGION)
ec2_resource = boto3.resource('ec2', region_name=REGION)
msk_client   = boto3.client('kafka')

- [Boto3 Docs: AWS MSK](https://boto3.amazonaws.com/v1/documentation/api/1.35.9/reference/services/kafka.html)

## [KnowledgeAmplifier: AWS MSK Service Setup](https://www.youtube.com/watch?v=BFKmQAafE_c&list=PLjfRmoYoxpNq-pjHW0n1AfkKNi_OOMUbi&index=1&t=685s)

<b style="color:red">Steps</b>: Double-click here.

<!--

Step 1:
-------
Cretae VPC -- Name -- virtual-private-cloud  IPv4 CIDR -- 10.0.0.0/16
Host address range -- 10.0.0.1 - 10.0.255.254

Step 2:
-------
Create 2 public subnets 
Public-Subnet-A--10.0.0.0/24
Host address range -- 10.0.0.1 - 10.0.0.254

Public-Subnet-B--10.0.1.0/24
Host address range -- 10.0.1.1 - 10.0.1.254

Step 3:
-------
Check the default route table -- you will see the above 2 subnets have not been explicitly associated with any route tables and are therefore associated with the main route table.

Step 4:
-------
Create a IGW & connect with VPC

Step 5:
-------
Add the IGW in default route table


Step 6:
-------
Launch MSK Cluster with vpc you created , unauthorised access allowed , plaintext enxryption
(keep security group as it is)

Step 7:
-------
Launch Linux EC2
In the list Network choose the VPC previously created.
In the list Auto-assign Public IP, choose Enable.

Step 8:
-------
Once the client for Amazon MSK has been created, the security group rules must be configured to allow the connection between the cluster and the client that we have just created.

For that , Add the security group id of ec2 to msk cluster security group all traffic

Repeat these steps to add an inbound rule in the security group that corresponds to your client computer to allow it to receive traffic from the security group from the VPC. Now your client computer can communicate bidirectionally with the MSK Cluster.

Once this is done, the newly created and configured client can be accessed.

Step 9:
-------
sudo yum install java-1.8.0-openjdk
wget https://archive.apache.org/dist/kafka/2.8.1/kafka_2.12-2.8.1.tgz
tar -xvf kafka_2.12-2.8.1.tgz
cd kafka_2.12-2.8.1

bin/kafka-topics.sh --create --topic demo_testing2 --bootstrap-server {Put the MSK bootstrap server URLs here} --replication-factor 1 --partitions 1
bin/kafka-topics.sh --create --topic helloworld --bootstrap-server {Put the MSK bootstrap server URLs here}  --replication-factor 1 --partitions 1

Step 10:
--------
Start the kafka Producer
---------------------------
bin/kafka-console-producer.sh --topic demo_testing2 --bootstrap-server {Put the MSK bootstrap server URLs here} 

In a new console start the kafka consumer--
cd kafka_2.12-2.8.1
bin/kafka-console-consumer.sh --topic helloworld --bootstrap-server {Put the MSK bootstrap server URLs here} 

Step 11:
--------
Install confluent kafka within kafka_2.12-2.8.1)
wget  http://packages.confluent.io/archive/5.1/confluent-5.1.2-2.11.zip
unzip confluent-5.1.2-2.11.zip

export CONFLUENT_HOME=/home/ec2-user/kafka_2.12-2.8.1/confluent-5.1.2
export PATH=$PATH:$CONFLUENT_HOME/bin
(Note , if installing confluent kafka , where kafka is installed (i.e. in /home/ec2-user) , then CONFLUENT_HOME should be -- /home/ec2-user/confluent-5.1.2)

Step 12:
--------
Change the bootstrap.servers in  confluent-5.1.2/etc/kafka-rest/kafka-rest.properties 



Step 13:
--------
Start Kafka Rest 
/home/ec2-user/kafka_2.12-2.8.1/confluent-5.1.2/bin/kafka-rest-start /home/ec2-user/kafka_2.12-2.8.1/confluent-5.1.2/etc/kafka-rest/kafka-rest.properties 

(Don't forget to allow all traffic to the security group of EC2 client machine)

Url to post messages using Kafka rest API--
http://{Put your cleint machine's Public IP here}:8082/topics/demo_testing2

Content-Type: application/vnd.kafka.json.v2+json

Sample Message:
-------------
{"records":[{"value":{"name": "testUser"}}]}

Start consumer to see the messages:
-----------------------------------
cd kafka_2.12-2.8.1
bin/kafka-console-consumer.sh --topic demo_testing2 --bootstrap-server {Put the MSK bootstrap server URLs here} 

Apache Kafka and Confluent Kafka are related but distinct in their offerings and use cases. Here's a comparison:

-   **Apache Kafka**: An open-source distributed event-streaming platform maintained by the Apache Software Foundation. It's freely available and is community-driven.

-   **Confluent Kafka**: Built on top of Apache Kafka by Confluent, a company founded by the creators of Kafka. Confluent enhances Kafka with additional tools, enterprise features, and managed services.

In [ ]:
AMAZON_LINUX_AMI = "ami-01816d07b1128cd2d"   # amazon Linux 2023 AMI
KAFKA_VERSION = '2.8.1'
KAFKA_DIR = f"""/home/ec2-user/kafka_2.12-{KAFKA_VERSION}"""
KAFKA_TOPIC_NAME = "httx-msk-topic" # demo_testing2

#### Prepare VPC with its components

In [ ]:
VPC_NAME = 'httx-msk-vpc'
vpc_cidr_block = '10.0.0.0/16'
VPC_ID = ec2_client.create_vpc(CidrBlock=vpc_cidr_block)['Vpc']['VpcId']

In [ ]:
# Add a Name tag to the VPC
ec2_client.create_tags(Resources=[VPC_ID], Tags=[{'Key': 'Name', 'Value': VPC_NAME}])
# ec2_client.describe_vpcs(VpcIds=[VPC_ID])

In [ ]:
subnet_configs = [
    {'cidr_block': '10.0.1.0/24', 'az': 'us-east-1a', 'tag': 'public-subnet-01'},
    {'cidr_block': '10.0.2.0/24', 'az': 'us-east-1b', 'tag': 'public-subnet-02'},
]

In [ ]:
public_subnet1 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[0]['cidr_block'],
    VpcId=VPC_ID,
    AvailabilityZone=subnet_configs[0]['az']
)
ec2_client.create_tags(Resources=[public_subnet1.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[0]['tag']}])

public_subnet2 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[1]['cidr_block'],
    VpcId=VPC_ID,
    AvailabilityZone=subnet_configs[1]['az']
)
ec2_client.create_tags(Resources=[public_subnet2.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[1]['tag']}])

In [ ]:
print(public_subnet1)

`NOTES:` The subnets are associated with the main route table since we havn't explicitly associated them with any route tables.

In [ ]:
# Create Internet Gateway and attach that with VPC
igw = ec2_resource.create_internet_gateway()
ec2_client.attach_internet_gateway(InternetGatewayId=igw.id, VpcId=VPC_ID)

In [ ]:
# Fetch all route tables associated with the VPC
route_tables = ec2_client.describe_route_tables(Filters=[{'Name': 'vpc-id','Values': [VPC_ID]}])
# Extract route table information
route_table_id = route_tables.get('RouteTables', [])[0]['Associations'][0]['RouteTableId']
print(route_table_id)

In [ ]:
route_params = {'DestinationCidrBlock': '0.0.0.0/0', 'GatewayId': igw.id}
ec2_client.create_route(RouteTableId=route_table_id, **route_params)   # Specify the Internet Gateway ID (optional)

#### Deploy MSK Cluster

In [ ]:
SECURITY_GROUP_ID_MSK_CLUSTER = ec2.create_security_group('MSK-CLUSTER-SG', VPC_ID)["GroupId"]

In [ ]:
MSK_CLUSTER_ARN = msk_client.create_cluster(
    ClusterName='httx-msk',  # Replace with your desired cluster name
    KafkaVersion='2.8.1',  # Replace with your desired Kafka version
    NumberOfBrokerNodes=2,  # Default number of brokers is 3
    BrokerNodeGroupInfo={
        'BrokerAZDistribution': 'DEFAULT',  # Distribute brokers across availability zones
        'InstanceType': 'kafka.t3.small', # Default broker instance type is 'kafka.m5.large' | ProvisionedThroughput is not supported for 'kafka.t3.small'
        'ClientSubnets': [
            public_subnet1.id,
            public_subnet2.id
        ],
        'SecurityGroups': [
            SECURITY_GROUP_ID_MSK_CLUSTER
        ],
        'StorageInfo': {
            'EbsStorageInfo': {
                # 'ProvisionedThroughput': {
                #     'Enabled': True,
                #     'VolumeThroughput': 250
                # },
                'VolumeSize': 10  # Default EBS volume size is 100 in GiB
            }
        }
    },
    # ConfigurationInfo={
    #     'Arn': 'string',
    #     'Revision': 123
    # },
    ClientAuthentication={
        # 'Sasl': {
        #     'Scram': {
        #         'Enabled': False
        #     },
        #     'Iam': {
        #         'Enabled': False
        #     }
        # },
        # 'Tls': {
        #     'CertificateAuthorityArnList': [
        #         'string',
        #     ],
        #     'Enabled': False
        # },
        'Unauthenticated': {
            'Enabled': True # Allow unauthorized access
        }
    },
    EncryptionInfo={
        # 'EncryptionAtRest': {         # by default available
        #     'DataVolumeKMSKeyId': 'string'
        # },
        'EncryptionInTransit': {
            'ClientBroker': 'TLS_PLAINTEXT',  # Encryption between clients and brokers (default is TLS)
            'InCluster': True
        }
    },
    EnhancedMonitoring='DEFAULT',  # Monitoring level
    OpenMonitoring={
        'Prometheus': {
            'JmxExporter': {
                'EnabledInBroker': False  # Default JMX exporter configuration
            },
            'NodeExporter': {
                'EnabledInBroker': False  # Default Node exporter configuration
            }
        }
    },
    LoggingInfo={
        'BrokerLogs': {
            'CloudWatchLogs': {
                'Enabled': False
            },
            'Firehose': {
                'Enabled': False
            },
            'S3': {
                'Enabled': False
            }
        }
    },
    Tags={
        'Environment': 'httx-test-MSK'  # Add your tags here
    }
)['ClusterArn']

In [ ]:
print(MSK_CLUSTER_ARN)

#### Run EC2 Instance

In [ ]:
SECURITY_GROUP_ID_MSK_CLIENT = ec2.create_security_group('MSK-CLIENT-SG', VPC_ID)["GroupId"]

In [ ]:
# Launch EC2 instance with tagging using TagSpecifications
MSK_CLIENT_INSTANCE_ID = ec2_client.run_instances(
    ImageId=AMAZON_LINUX_AMI,  # amazon Linux 2023 AMI
    InstanceType='t2.micro',          #'t2.medium', 't2.micro'
    MinCount=1,
    MaxCount=1,
    KeyName='AMominNJ',               # Replace with your key pair
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [{'Key': 'Name', 'Value': 'KAFKA_CLIENT'}]
        }
    ],
    BlockDeviceMappings=[
        {
            'DeviceName': '/dev/sda1',     # Default root volume
            'Ebs': {
                'VolumeSize': 10,          # Volume size in GiB
                'VolumeType': 'gp2'        # General Purpose SSD
            }
        }
    ],
    SecurityGroupIds=[SECURITY_GROUP_ID_MSK_CLIENT],
    SubnetId=public_subnet1.id
)['Instances'][0]['InstanceId']

In [ ]:
MSK_CLIENT_INSTANCE_ID='i-0e235b083a673ab1c'

In [ ]:
# # Start the instance
# ec2_client.start_instances(InstanceIds=[MSK_CLIENT_INSTANCE_ID])

In [ ]:
response = ec2_client.describe_instances(InstanceIds=[MSK_CLIENT_INSTANCE_ID])
# print(response)
# print(response['Reservations'][0]['Instances'][0]['PublicDnsName'])
# print(response['Reservations'][0]['Instances'][0]['PublicIpAddress'])
CLIENT_PUBLIC_IP = response['Reservations'][0]['Instances'][0]['PublicIpAddress']

In [ ]:
# Adds an inbound rule to SECURITY_GROUP_ID_MSK_CLIENT to allow all traffics from SECURITY_GROUP_ID_MSK_CLUSTER
ec2_client.authorize_security_group_ingress(
    GroupId=SECURITY_GROUP_ID_MSK_CLIENT,
    IpPermissions=[
        {
            'IpProtocol': '-1',  # '-1' means all protocols
            'UserIdGroupPairs': [
                {
                    'GroupId': SECURITY_GROUP_ID_MSK_CLUSTER,
                    'Description': 'Allow all traffic from SECURITY_GROUP_ID_MSK_CLUSTER security group'
                }
            ]
        }
    ]
)

In [ ]:
# Adds an inbound rule to SECURITY_GROUP_ID_MSK_CLUSTER to allow all traffics from ALL_IN_ONE_SG
ec2_client.authorize_security_group_ingress(
    GroupId=SECURITY_GROUP_ID_MSK_CLUSTER,
    IpPermissions=[
        {
            'IpProtocol': '-1',  # '-1' means all protocols
            'UserIdGroupPairs': [
                {
                    'GroupId': SECURITY_GROUP_ID_MSK_CLIENT,
                    'Description': 'Allow all traffic from SECURITY_GROUP_ID_MSK_CLIENT security group'
                }
            ]
        }
    ]
)

#### Configure Kakfa Client Instance

In [ ]:
# ! ssh amazon_linux 'sudo yum install java-1.8.0-openjdk'
! ssh amazon_linux 'sudo yum -y install java-11'

In [ ]:
! ssh amazon_linux 'wget https://archive.apache.org/dist/kafka/{KAFKA_VERSION}/kafka_2.12-{KAFKA_VERSION}.tgz'

In [ ]:
! ssh amazon_linux 'tar -xvf kafka_2.12-{KAFKA_VERSION}.tgz'

In [ ]:
BOOTSTRAP_SERVERS_ENDPOINT=msk_client.get_bootstrap_brokers(ClusterArn=MSK_CLUSTER_ARN)['BootstrapBrokerString']
print(BOOTSTRAP_SERVERS_ENDPOINT)

In [ ]:
# Create Kafka Topic: `$ ssh amazon_linux`
command1 = f"""{KAFKA_DIR}/bin/kafka-topics.sh --create --topic {KAFKA_TOPIC_NAME} --bootstrap-server {BOOTSTRAP_SERVERS_ENDPOINT} --replication-factor 1 --partitions 1"""
print(command1)

In [ ]:
# Start the kafka Producer: `$ ssh amazon_linux`
command3 = f"""{KAFKA_DIR}/bin/kafka-console-producer.sh --topic {KAFKA_TOPIC_NAME} --bootstrap-server {BOOTSTRAP_SERVERS_ENDPOINT}"""
print(command3)

In [ ]:
# In a new console start the kafka consumer: `$ ssh amazon_linux`
command4 = f"""{KAFKA_DIR}/bin/kafka-console-consumer.sh --topic {KAFKA_TOPIC_NAME} --bootstrap-server {BOOTSTRAP_SERVERS_ENDPOINT}"""
print(command4)

-   **Download, Install and Configure Confluent Kafk**:
    -   It facilitate publishing message into Kafka Topic using REST API

In [ ]:
! ssh amazon_linux 'wget -P {KAFKA_DIR}/ http://packages.confluent.io/archive/5.1/confluent-5.1.2-2.11.zip'

In [ ]:
! ssh amazon_linux 'unzip {KAFKA_DIR}/confluent-5.1.2-2.11.zip -d {KAFKA_DIR}/'

In [ ]:
import subprocess

strings = r"""
export CONFLUENT_HOME=/home/ec2-user/kafka_2.12-2.8.1/confluent-5.1.2
export PATH=$PATH:$CONFLUENT_HOME/bin
"""

command7 = f"""echo '{strings}' | ssh amazon_linux 'cat >> /home/ec2-user/.bashrc'"""
subprocess.run(command7, shell=True)

In [ ]:
print(BOOTSTRAP_SERVERS_ENDPOINT.split(","))

In [ ]:
BROCKER1_ENDPOINT=BOOTSTRAP_SERVERS_ENDPOINT.split(",")[0]
BROCKER2_ENDPOINT=BOOTSTRAP_SERVERS_ENDPOINT.split(",")[1]
prefix = "bootstrap.servers=PLAINTEXT://localhost:9092"
replacement = f"""bootstrap.servers=PLAINTEXT://{BROCKER1_ENDPOINT},PLAINTEXT://{BROCKER2_ENDPOINT}"""
print(replacement)

In [ ]:
## NOT DONE YET
# command = f"""sed -i '/^{prefix}/c\{replacement}' $CONFLUENT_HOME/confluent-5.1.2/etc/kafka-rest/kafka-rest.properties"""
# print(command)


In [ ]:
%%bash
# Your bash commands go here
echo "This is a bash cell"
ls -l


-   Edit `confluent-5.1.2/etc/kafka-rest/kafka-rest.properties` file
    -   Replace `old_text` by `new_text`

In [ ]:
## Start Kafka Rest API Server
command4 = f"""sudo {KAFKA_DIR}/confluent-5.1.2/bin/kafka-rest-start {KAFKA_DIR}/confluent-5.1.2/etc/kafka-rest/kafka-rest.properties"""
print(command4)

In [ ]:
CLIENT_PUBLIC_IP='54.90.206.152'

## Url to post messages using Kafka rest API--
KAFKA_REST_ENDPOINT = f"""http://{CLIENT_PUBLIC_IP}:8082/topics/{KAFKA_TOPIC_NAME}"""

"""Content-Type: application/vnd.kafka.json.v2+json"""

# Sample Message:
{"records":[{"value":{"name": "testUser"}}]}

In [ ]:
def publish_to_kafka_rest(endpoint: str, message: dict, headers: dict = None):
    """
    Publishes a message to a Kafka topic using the Kafka REST API.

    Parameters:
    - endpoint (str): The Kafka REST endpoint, e.g., http://<CLIENT_PUBLIC_IP>:8082/topics/<KAFKA_TOPIC_NAME>
    - message (dict): The message to publish, formatted as JSON.
    - headers (dict): Optional headers for the HTTP request. Default is None.

    Returns:
    - dict: The response from the Kafka REST API.
    """
    # Default headers
    if headers is None:
        headers = {
            "Content-Type": "application/vnd.kafka.json.v2+json"
        }
    
    try:
        # POST request to the Kafka REST API
        response = requests.post(endpoint, json={"records": [{"value": message}]}, headers=headers)
        
        # Raise an HTTPError if the response status is not successful
        response.raise_for_status()
        
        return response.json()  # Parse JSON response
    
    except requests.exceptions.RequestException as e:
        print(f"Error publishing to Kafka: {e}")
        return {"error": str(e)}

message = {
    "key": "example-key",
    "value": {
        "field1": "value1",
        "field2": "value2"
    }
}

publish_to_kafka_rest(KAFKA_REST_ENDPOINT, message)

In [ ]:
### Start consumer to see the messages: `$ ssh amazon_linux`
command4 = f"""{KAFKA_DIR}/bin/kafka-console-consumer.sh --topic {KAFKA_TOPIC_NAME} --bootstrap-server {BOOTSTRAP_SERVERS_ENDPOINT}"""
print(command4)

#### API Gateway Integration with Kafka Client [`NOT COMPLETED`]

<div style="text-align:center" ><img src="./ApiGatewayIntegrationWithKafkaClient.png" width="500" height="300" /></div>

#### Delete Resources

In [ ]:
ec2.remove_all_rules(SECURITY_GROUP_ID_MSK_CLUSTER)
ec2.remove_all_rules(SECURITY_GROUP_ID_MSK_CLIENT)

In [ ]:
vpc.delete_vpc_with_dependencies(VPC_ID)

In [ ]:
msk_client.delete_cluster(ClusterArn=MSK_CLUSTER_ARN)

In [ ]:
# # Stop the instance immediately after creation
ec2_client.terminate_instances(InstanceIds=[MSK_CLIENT_INSTANCE_ID])

## [Setup Amazon MSK (Kafka) as an event source for Lambda](https://www.youtube.com/watch?v=RGGLBEDUuMc)

![](./msk_lambda.png)

<b style="color:red">Steps</b>: Double-click here.

<!--

- Install java on client machine.
   ```
    sudo yum -y install java-11
   ```


- Download Apache Kafka.
  ```
   wget https://archive.apache.org/dist/kafka/{YOUR MSK VERSION}/kafka_2.13-{YOUR MSK VERSION}.tgz
  ```


- Run the following command in the directory where you downloaded the TAR file in the previous step.
  ```
  tar -xzf kafka_2.13-{YOUR MSK VERSION}.tgz
  ```


- Go to the kafka_2.13-{YOUR MSK VERSION}/libs directory, then run the following command to download the Amazon MSK IAM JAR file.
  ```
  wget https://github.com/aws/aws-msk-iam-auth/releases/download/v1.1.1/aws-msk-iam-auth-1.1.1-all.jar
  ```


- Go to the kafka_2.13-{YOUR MSK VERSION}/bin directory. Copy the following property settings and paste them into a new file. Name the file client.properties and save it.
    ```
    security.protocol=SASL_SSL
    sasl.mechanism=AWS_MSK_IAM
    sasl.jaas.config=software.amazon.msk.auth.iam.IAMLoginModule required;
    sasl.client.callback.handler.class=software.amazon.msk.auth.iam.IAMClientCallbackHandler
    ```

- To get the broker list, run following command:
  ```
  aws kafka get-bootstrap-brokers --cluster-arn CLUSTER_ARN
  ```

 
- Create the Topic, run the following command, replacing BootstrapServerString with one of the broker endpoints that you obtained in the previous step.
  ```
  <path-to-your-kafka-installation>/bin/kafka-topics.sh --create --bootstrap-server BootstrapServerString --command-config client.properties --replication-factor 2 --partitions 1 --topic MSKTutorialTopic
  ```

 
- Producer Command:
  ```
  /home/ec2-user/kafka_2.13-3.5.1/bin/kafka-console-producer.sh --broker-list BROKER_LIST --producer.config client.properties --topic MSKTutorialTopic
  ```

 
- Consumer Command:
  ```
  /home/ec2-user/kafka_2.13-3.5.1/bin/kafka-console-consumer.sh --bootstrap-server BROKER_LIST --consumer.config client.properties --topic MSKTutorialTopic --from-beginning
  ```


- Execution role for Lambda function : `AWSLambdaMSKExecutionrole`

- **Reference documents**:
  - [Kafka event example](https://docs.aws.amazon.com/lambda/latest/dg/with-msk.html)
  - [Client machine steps](https://docs.aws.amazon.com/msk/latest/developerguide/create-client-machine.html)

In [ ]:
EC2_POLICY = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "EC2MSKRoleId0",
      "Effect": "Allow",
      "Action": [
        "kafka-cluster:*Topic*",
        "kafka-cluster:AlterGroup",
        "kafka-cluster:ReadData",
        "kafka-cluster:DescribeCluster",
        "kafka-cluster:AlterCluster",
        "kafka-cluster:DescribeTopic",
        "kafka:Update*",
        "kafka-cluster:DescribeGroup",
        "kafka-cluster:Connect",
        "kafka-cluster:WriteData",
        "kafka:GetBootstrapBrokers"
      ],
      "Resource": "*"
    }
  ]
}

LAMBDA_POLICY = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "kafka-cluster:Connect",
        "kafka-cluster:DescribeGroup",
        "kafka-cluster:AlterGroup",
        "kafka-cluster:DescribeTopic",
        "kafka-cluster:ReadData",
        "kafka-cluster:DescribeClusterDynamicConfiguration"
      ],
      "Resource": [
        "arn:aws:kafka:ap-east-1:123456789123:cluster/demo-msk-cluster/ab7c0f32-n123-4567-8r23-12a3c04z5k8p-7",
        "arn:aws:kafka:ap-east-1:123456789123:topic/demo-msk-cluster/ab7c0f32-n123-4567-8r23-12a3c04z5k8p-7/*",
        "arn:aws:kafka:ap-east-1:123456789123:group/demo-msk-cluster/ab7c0f32-n123-4567-8r23-12a3c04z5k8p-7/*"
      ]
    }
  ]
}

In [21]:
KAFKA_VERSION='3.5.1'

In [ ]:
# amazon Linux 2023 AMI
BASIC_INSTANCE_ID = ec2.run_ec2_instance(image_id='ami-01816d07b1128cd2d')

In [ ]:
response = ec2_client.describe_instances(InstanceIds=[BASIC_INSTANCE_ID])
print(response['Reservations'][0]['Instances'][0]['PublicDnsName'])
print(response['Reservations'][0]['Instances'][0]['PublicIpAddress'])
print(response['Reservations'][0]['Instances'][0]['InstanceId'])

In [ ]:
! ssh amazon 'sudo yum -y install java-11'

In [ ]:
! ssh amazon 'wget https://archive.apache.org/dist/kafka/{KAFKA_VERSION}/kafka_2.13-{KAFKA_VERSION}.tgz'

In [17]:
! ssh amazon 'tar -xzf kafka_2.13-{KAFKA_VERSION}.tgz'

In [ ]:
! ssh amazon 'wget -P /home/ec2-user/kafka_2.13-{KAFKA_VERSION}/libs https://github.com/aws/aws-msk-iam-auth/releases/download/v1.1.1/aws-msk-iam-auth-1.1.1-all.jar'

In [23]:
! ssh amazon 'touch /home/ec2-user/kafka_2.13-{KAFKA_VERSION}/bin/client.properties'

In [ ]:
import subprocess

strings = r"""security.protocol=SASL_SSL
sasl.mechanism=AWS_MSK_IAM
sasl.jaas.config=software.amazon.msk.auth.iam.IAMLoginModule required;
sasl.client.callback.handler.class=software.amazon.msk.auth.iam.IAMClientCallbackHandler"""

command = f"""echo '{strings}' | ssh amazon 'cat >> /home/ec2-user/kafka_2.13-{KAFKA_VERSION}/bin/client.properties'"""
subprocess.run(command, shell=True)

#### Lambda


```js
exports.handler = async (event) => {
    for (let key in event.records) {
      console.log('Key: ', key)

      event.records[key].map((record) => {
        console.log('Record: ', record)
        const msg = Buffer.from(record.value, 'base64').toString()
        console.log('Message:', msg)
      })
    }
}
```

#### Delete Resources

In [ ]:
ec2_client.terminate_instances(InstanceIds=[BASIC_INSTANCE_ID])